In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada.communicator import SocketCommunicator

def main(communicator):
    print(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

Hello from player 1!Hello from player 0!



In [3]:
logging.getLogger("cicada.communicator").setLevel(logging.INFO)

SocketCommunicator.run(world_size=2, fn=main)

logging.getLogger("cicada.communicator").setLevel(logging.WARNING) # Put things back the way they were

INFO:cicada.communicator.socket.connect:Comm world player 0 listening to tcp://127.0.0.1:53647 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 listening to tcp://127.0.0.1:53649 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 direct connect with ['tcp://127.0.0.1:53647', 'tcp://127.0.0.1:53649'].
INFO:cicada.communicator.socket.connect:Comm world player 0 direct connect with ['tcp://127.0.0.1:53647', 'tcp://127.0.0.1:53649'].
INFO:cicada.communicator.socket.connect:Comm world player 1 tcp://127.0.0.1:53653 connected to player 0 tcp://127.0.0.1:53647
INFO:cicada.communicator.socket.connect:Comm world player 0 tcp://127.0.0.1:53647 accepted connection from tcp://127.0.0.1:53653
INFO:cicada.communicator.socket:Comm world player 0 communicator ready.
INFO:cicada.communicator.socket:Comm world player 1 communicator ready.


Hello from player 1!Hello from player 0!



INFO:cicada.communicator.socket:Comm world player 0 communicator freed.
INFO:cicada.communicator.socket:Comm world player 1 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 result: None
INFO:cicada.communicator.socket:Comm world player 1 result: None


In [4]:
from cicada.logging import Logger

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0!
INFO:root:Hello from player 1!


In [5]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.warning(f"Warning message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

In [6]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.error(f"Error message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

ERROR:root:Error message from player 0!
ERROR:root:Error message from player 1!


In [7]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.critical(f"Critical message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

CRITICAL:root:Critical message from player 0!
CRITICAL:root:Critical message from player 1!


In [8]:
logging.getLogger().setLevel(logging.DEBUG) # Don't filter DEBUG messages

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.debug(f"Debug message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

DEBUG:root:Debug message from player 0!
DEBUG:root:Debug message from player 1!


In [9]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=1)

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Hello only from player 1!


In [10]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=[0, 2, 4])

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Hello only from player 0!
INFO:root:Hello only from player 2!
INFO:root:Hello only from player 4!


In [11]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator, sync=False)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 1!
INFO:root:Hello from player 0!


In [12]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    
    try:
        log.info(f"Player {communicator.rank} starting ...")
        
        # Wait for a message that never arrives ...
        if communicator.rank == 1:
            payload = communicator.recv(src=0, tag=42)
        
    except Exception as e:
        log.error(f"Player {communicator.rank} exception: {e}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 starting ...
INFO:root:Player 1 starting ...
ERROR:cicada.communicator.socket:Comm world player 1 failed: Timeout('Tag LOGSYNC from player 0 timed-out after 5s')


In [13]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    
    try:
        log.info(f"Player {communicator.rank} starting ...")
        
        # Wait for a message that will neve arrives
        if communicator.rank == 0:
            payload = communicator.recv(src=1, tag=42)
        
    except Exception as e:
        log.sync = False # Disable coordinated logging
        log.error(f"Player {communicator.rank} exception: {e}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 starting ...
INFO:root:Player 1 starting ...
ERROR:root:Player 0 exception: Tag 42 from player 1 timed-out after 5s


In [14]:
logging.getLogger("cicada.communicator.socket.transcript").setLevel(logging.DEBUG)

In [15]:
import numpy

from cicada.additive import AdditiveProtocolSuite

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)
    
SocketCommunicator.run(fn=main, world_size=3);

DEBUG:cicada.communicator.socket.transcript:Comm world player 1 --> player 2 PRSZ 6492288262783034151
DEBUG:cicada.communicator.socket.transcript:Comm world player 0 --> player 1 PRSZ 6361396029439371230
DEBUG:cicada.communicator.socket.transcript:Comm world player 2 --> player 0 PRSZ 8651648179616350831
DEBUG:cicada.communicator.socket.transcript:Comm world player 0 <-- player 2 PRSZ 8651648179616350831
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 <-- player 0 PRSZ 6361396029439371230
DEBUG:cicada.communicator.socket.transcript:Comm world player 2 <-- player 1 PRSZ 6492288262783034151
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 --> player 0 GATHER 13499671322104102560
DEBUG:cicada.communicator.socket.transcript:Comm world player 0 --> player 0 GATHER 6348686854485304612
DEBUG:cicada.communicator.socket.transcript:Comm world player 2 --> player 0 GATHER 17045129970830023622
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 --> player

If you want to customize the transcript, you can use a rich set of extra logging fields in your formatting string including: `arrow` (message direction, relative to the local rank), `comm` (communicator name), `dir` (message direction, relative to the local rank), `dst` (message destination), `other` (the other player communicating with the local rank), `payload` (message payload), `rank` (local player), `src` (message source), `tag` (message type), and `verb` (whether the message is "send" or "receive", relative to the local player).

For example, if you wanted to change the default formatting to produce a more compact version of the above, you could do the following:

In [16]:
logging.getLogger().handlers[0].setFormatter(logging.Formatter("Player {rank}: {rank} {dir} {other} {tag} {payload}", style="{"))

SocketCommunicator.run(fn=main, world_size=3);

Player 0: 0 > 1 PRSZ 6264498258166304758
Player 2: 2 > 0 PRSZ 8034178986254176938
Player 1: 1 > 2 PRSZ 1723687555232128984
Player 0: 0 < 2 PRSZ 8034178986254176938
Player 2: 2 < 1 PRSZ 1723687555232128984
Player 1: 1 < 0 PRSZ 6264498258166304758
Player 2: 2 > 0 GATHER 23076347000739288
Player 0: 0 > 0 GATHER 9532638137798066714
Player 1: 1 > 0 GATHER 8891029588911073235
Player 0: 0 < 0 GATHER 9532638137798066714
Player 2: 2 > 1 GATHER 23076347000739288
Player 1: 1 > 1 GATHER 8891029588911073235
Player 0: 0 < 2 GATHER 23076347000739288
Player 2: 2 > 2 GATHER 23076347000739288
Player 1: 1 < 1 GATHER 8891029588911073235
Player 0: 0 < 1 GATHER 8891029588911073235
Player 2: 2 < 2 GATHER 23076347000739288
Player 1: 1 < 2 GATHER 23076347000739288
Player 0: 0 > 1 GATHER 9532638137798066714
Player 0: 0 > 2 GATHER 9532638137798066714
Player 1: 1 < 0 GATHER 9532638137798066714
Player 2: 2 < 0 GATHER 9532638137798066714
Player 1: 1 > 2 GATHER 8891029588911073235
Player 2: 2 < 1 GATHER 889102958891

Note that the default transcript includes entries for the transmission of a message by the sender along with its arrival at the destination, and that the rank of the local player is always on the left, whether they are sending or receiving.  This is reasonably intuitive to look at, but it means that the role of the ranks and message direction aren't consistent.  As an alternative, it may be better in some cases to always put the sender on the left and the recipient on the right:

In [17]:
logging.getLogger().handlers[0].setFormatter(logging.Formatter("Player {rank}: {src} > {dst} {tag} {payload}", style="{"))

SocketCommunicator.run(fn=main, world_size=3);

Player 0: 0 > 1 PRSZ 7173715824903447528
Player 1: 1 > 2 PRSZ 3846579223251816653
Player 2: 2 > 0 PRSZ 6303663491872761811
Player 1: 0 > 1 PRSZ 7173715824903447528
Player 2: 1 > 2 PRSZ 3846579223251816653
Player 0: 2 > 0 PRSZ 6303663491872761811
Player 1: 1 > 0 GATHER 13960316688736225276
Player 2: 2 > 0 GATHER 12801492427416139792
Player 0: 0 > 0 GATHER 10131679031267065726
Player 1: 1 > 1 GATHER 13960316688736225276
Player 2: 2 > 1 GATHER 12801492427416139792
Player 0: 1 > 0 GATHER 13960316688736225276
Player 0: 0 > 0 GATHER 10131679031267065726
Player 1: 1 > 1 GATHER 13960316688736225276
Player 2: 2 > 2 GATHER 12801492427416139792
Player 0: 2 > 0 GATHER 12801492427416139792
Player 1: 2 > 1 GATHER 12801492427416139792
Player 2: 2 > 2 GATHER 12801492427416139792
Player 0: 0 > 1 GATHER 10131679031267065726
Player 0: 0 > 2 GATHER 10131679031267065726
Player 1: 0 > 1 GATHER 10131679031267065726
Player 1: 1 > 2 GATHER 13960316688736225276
Player 2: 0 > 2 GATHER 10131679031267065726
Player

Since each message appears twice in the transcript (once when sent, and once when received), you may wish to filter the output to eliminate the duplication, e.g. by only logging messages when they're sent:

In [18]:
def only_sent(record):
    return record.verb == "send"

logging.getLogger().handlers[0].addFilter(only_sent)

SocketCommunicator.run(fn=main, world_size=3);

Player 0: 0 > 1 PRSZ 1667230873029740010
Player 2: 2 > 0 PRSZ 577449036838102253
Player 1: 1 > 2 PRSZ 173127242338774306
Player 1: 1 > 0 GATHER 16258864812699647689
Player 2: 2 > 0 GATHER 7183570836399356046
Player 0: 0 > 0 GATHER 13451052498320427059
Player 1: 1 > 1 GATHER 16258864812699647689
Player 2: 2 > 1 GATHER 7183570836399356046
Player 0: 0 > 1 GATHER 13451052498320427059
Player 2: 2 > 2 GATHER 7183570836399356046
Player 0: 0 > 2 GATHER 13451052498320427059
Player 1: 1 > 2 GATHER 16258864812699647689
